In [233]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM 
from datetime import datetime
from sklearn.svm import OneClassSVM 

In [234]:
week_1 = pd.read_csv('registro_transazioni_week1.csv', sep=';', engine = 'python')
week_2 = pd.read_csv('registro_transazioni_week2.csv', sep=';', engine = 'python')
week_3 = pd.read_csv('registro_transazioni_week3.csv', sep=';', engine = 'python')
week_4 = pd.read_csv('registro_transazioni_week4.csv', sep=';', engine = 'python')

In [235]:
week_1['DATA_E_ORA'] = pd.to_datetime(week_1['DATA_E_ORA'])
week_2['DATA_E_ORA'] = pd.to_datetime(week_2['DATA_E_ORA'])
week_3['DATA_E_ORA'] = pd.to_datetime(week_3['DATA_E_ORA'])
week_4['DATA_E_ORA'] = pd.to_datetime(week_4['DATA_E_ORA'])

In [236]:
week_2

,PARTITAIVA,FARMACO,QUANTITA,DATA_E_ORA
0,02631650427,CLINDAMICINA IBI,3,2020-03-08 00:00:34
1,01054480429,FALEV,4,2020-03-08 00:00:38
2,02204580423,ATENOLOLO ALMUS,4,2020-03-08 00:00:41
3,01430880425,SAIDOX,2,2020-03-08 00:00:43
4,00109450429,FUROSEMIDE DOC GENERICI,2,2020-03-08 00:01:05
...,...,...,...,...
62764,02483570426,RAMIPRIL SANDOZ,5,2020-03-14 23:58:38
62765,00194530424,DORYX,3,2020-03-14 23:58:39
62766,00887830420,PRAVASTATINA ARISTO,1,2020-03-14 23:59:13
62767,00992280420,PANTORC,2,2020-03-14 23:59:56


In [237]:
lista_farmacie = week_1['PARTITAIVA'].unique()
lista_farmacie

array(['02355240421', '00155670425', '02721500425', '03664141201',
       '00900100421', '02516810427', '00066360421', '01229120421',
       '01304610429', '02769570421', '01541420426', '01063290421',
       '00108460429', '01574740427', '02502470426', '02549300420',
       '02552120426', '02412410421', '00162290423', '00992560425',
       '02804550420', '00045190428', '01304600420', '00936580422',
       '00176190429', '00045360427', '02513010427', '02542140427',
       '02803660428', '01577040429', '01168600425', '00387550429',
       '02067420428', '00171160427', '00156300428', '02006840421',
       '02374750428', '00135200426', '02310740424', '00817700420',
       '00094720422', '01430880425', '02834420420', '02354140424',
       '00194530424', '00156670424', '01977970431', '00106410426',
       '00008080426', '01327770424', '00032630428', '00935300426',
       '01325090429', '01165430420', '01591190424', '02535540427',
       '00035600428', '02310080425', '02077490429', '013311604

In [239]:
def genera_df(df,inizio,fine):
    for i in range(inizio,fine+1):
        df[str(i)] = 0
    return df

In [240]:
df_week_1 = pd.DataFrame(np.concatenate((np.reshape(lista_farmacie,(222,1)),np.ones((222,1))),axis=1)
                         , columns=['PARTITAIVA','WEEK_ID'])
df_week_2 = pd.DataFrame(np.concatenate((np.reshape(lista_farmacie,(222,1)),2*np.ones((222,1))),axis=1)
                         , columns=['PARTITAIVA','WEEK_ID'])
df_week_3 = pd.DataFrame(np.concatenate((np.reshape(lista_farmacie,(222,1)),3*np.ones((222,1))),axis=1)
                         , columns=['PARTITAIVA','WEEK_ID'])
df_week_4 = pd.DataFrame(np.concatenate((np.reshape(lista_farmacie,(222,1)),4*np.ones((222,1))),axis=1)
                         , columns=['PARTITAIVA','WEEK_ID'])

In [241]:
set_week_1 = genera_df(df_week_1,1,168)
set_week_2 = genera_df(df_week_2,1,168)
set_week_3 = genera_df(df_week_3,1,168)
set_week_4 = genera_df(df_week_4,1,168)

In [287]:
def calcolo_diff(df,week, week_id):
    codice = df['PARTITAIVA']
    print(codice)
    query = week[week['PARTITAIVA'] == codice]
    print(query)
    for i in range(1,8):
        giorno = query[query['DATA_E_ORA'].dt.day == i+(week_id-1)*7]
        print(giorno)
        for k in range(0,24):
            ora_prec = (giorno[giorno['DATA_E_ORA'].dt.hour == k])
            ora = (giorno[giorno['DATA_E_ORA'].dt.hour == k+1])
            diff = len(ora)-len(ora_prec)
            print(diff)
            df[str(24*(i-1)+(k+1))] = diff
    return df

In [294]:
def calcolo_diff(df,week, week_id):
    codice = df['PARTITAIVA']
    print(codice)
    query = week[week['PARTITAIVA'] == codice]
    print(query)
    for i in range(1,8):
        giorno = query[query['DATA_E_ORA'].dt.day == i+(week_id-1)*7]
        domani = query[query['DATA_E_ORA'].dt.day == i+(week_id-1)*7+1]
        #print(giorno)
        for k in range(0,24):
            if k != 23:
                ora_prec = (giorno[giorno['DATA_E_ORA'].dt.hour == k])
                ora = (giorno[giorno['DATA_E_ORA'].dt.hour == k+1])
                diff = len(ora)-len(ora_prec)
                #print(diff)
                df[str(24*(i-1)+(k+1))] = diff
            else:
                ora_prec = (giorno[giorno['DATA_E_ORA'].dt.hour == k])
                ora = (domani[domani['DATA_E_ORA'].dt.hour == 0])
                diff = len(ora)-len(ora_prec)
                #print(diff)
                df[str(24*(i-1)+(k+1))] = diff
    return df       

In [281]:
calcolo_diff_2(set_week_4,'02310740424',week_4,4)

['02310740424']
        PARTITAIVA                                      FARMACO  QUANTITA  \
37     02310740424                     CIPROFLOXACINA AUROBINDO         2   
227    02310740424                              FOSFOMICINA SUN         1   
541    02310740424                                        ACTOS         2   
548    02310740424                           CAPECITABINA MYLAN         5   
608    02310740424                                       IBUMAL         4   
...            ...                                          ...       ...   
61836  02310740424                             RIVASTIGMINA DOC         4   
61927  02310740424  CANDESARTAN E IDROCLOROTIAZIDE EUROGENERICI         4   
62103  02310740424                      ROSUVASTATINA AUROBINDO         2   
62496  02310740424                                     PARLODEL         1   
62675  02310740424                                    MIRAPEXIN         2   

               DATA_E_ORA  
37    2020-03-22 00:08:51  
227

<ipython-input-280-62e36db11b16>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(i*(k+1))] = diff
<ipython-input-280-62e36db11b16>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(i*(k+1))] = diff
<ipython-input-280-62e36db11b16>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

0
2
-2
0
0
0
0
0
0
0
1
0
-1
0
1
0
0
0
0
0
-1
0
0
-1
0
1
-1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
-1
1
-1
0
0
-1
0
1
-1
0
0
0
0
0
0
1
-1
1
-1
0
1
-1
1
-1
0
1
-1
0
0
-1


<ipython-input-280-62e36db11b16>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(i*(k+1))] = diff
<ipython-input-280-62e36db11b16>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(i*(k+1))] = diff
<ipython-input-280-62e36db11b16>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

,PARTITAIVA,WEEK_ID,1,2,3,4,5,6,7,8,...,159,160,161,162,163,164,165,166,167,168
38,02310740424,1,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,-1


In [295]:
final_df_week_1 = set_week_1.apply(calcolo_diff,week = week_1, week_id = 1, axis=1)
final_df_week_2 = set_week_2.apply(calcolo_diff,week = week_2, week_id = 2, axis=1)
final_df_week_3 = set_week_3.apply(calcolo_diff,week = week_3, week_id = 3, axis=1)
final_df_week_4 = set_week_4.apply(calcolo_diff,week = week_4, week_id = 4, axis=1)

02355240421
        PARTITAIVA                          FARMACO  QUANTITA  \
0      02355240421  LOSARTAN E IDROCLOROTIAZIDE SUN         1   
379    02355240421                        MIRAPEXIN         1   
526    02355240421       IDROCLOROTIAZIDE AUROBINDO         1   
775    02355240421          FEBUXOSTAT DOC GENERICI         4   
957    02355240421                         CIPROXIN         1   
...            ...                              ...       ...   
62013  02355240421                RASAGILINA ARISTO         3   
62027  02355240421                ARIPIPRAZOLO KRKA         5   
62418  02355240421                      METOPROLOLO         3   
62652  02355240421             LEVOFLOXACINA ACCORD         4   
62858  02355240421                          KAFENAC         3   

               DATA_E_ORA  
0     2020-03-01 00:00:04  
379   2020-03-01 01:44:08  
526   2020-03-01 02:27:13  
775   2020-03-01 03:26:55  
957   2020-03-01 04:16:15  
...                   ...  
62013 2020-

01304610429
        PARTITAIVA                    FARMACO  QUANTITA          DATA_E_ORA
8      01304610429                    DILIVAS         5 2020-03-01 00:01:08
282    01304610429           LANSOPRAZOLO GIT         1 2020-03-01 01:15:37
493    01304610429                   AVERELIX         2 2020-03-01 02:17:14
836    01304610429  MOXONIDINA MYLAN GENERICS         1 2020-03-01 03:44:19
1075   01304610429                   ADENURIC         1 2020-03-01 04:52:03
...            ...                        ...       ...                 ...
61860  01304610429             QUETIAPINA DOC         1 2020-03-07 20:08:22
61943  01304610429            VALACICLOVIR EG         2 2020-03-07 20:20:32
62270  01304610429       ATORVASTATINA PFIZER         5 2020-03-07 21:13:03
62618  01304610429     GLICLAZIDE ZENTIVA LAB         4 2020-03-07 22:45:34
62768  01304610429                    MAZDIMA         3 2020-03-07 23:28:40

[285 rows x 4 columns]
02769570421
        PARTITAIVA                      

02552120426
        PARTITAIVA                 FARMACO  QUANTITA          DATA_E_ORA
16     02552120426               BLUMIRTAX         3 2020-03-01 00:03:03
308    02552120426          OMEPRAZOLO SUN         1 2020-03-01 01:23:58
614    02552120426       ATORVASTATINA ABC         1 2020-03-01 02:47:19
719    02552120426       FOSFOMICINA MYLAN         3 2020-03-01 03:14:06
1027   02552120426  CINACALCET SANDOZ GMBH         4 2020-03-01 04:40:43
...            ...                     ...       ...                 ...
62002  02552120426                 VELAMOX         4 2020-03-07 20:29:20
62209  02552120426  PIOGLITAZONE AUROBINDO         5 2020-03-07 20:58:21
62309  02552120426                  TRAKOR         2 2020-03-07 21:22:06
62666  02552120426                 CEFODOX         4 2020-03-07 22:59:01
62853  02552120426                 ACEQUIN         2 2020-03-07 23:50:59

[284 rows x 4 columns]
02412410421
        PARTITAIVA              FARMACO  QUANTITA          DATA_E_ORA
17    

00176190429
        PARTITAIVA                           FARMACO  QUANTITA  \
24     00176190429                           PROSCAR         5   
229    00176190429                             ZIMOX         4   
542    00176190429                   SERTRALINA KRKA         4   
884    00176190429           DORZOLAMIDE TIMOLOLO EG         4   
1017   00176190429  ZOFENOPRIL E IDROCLOROTIAZIDE EG         5   
...            ...                               ...       ...   
62042  00176190429                   PREDNISONE TEVA         4   
62071  00176190429                    FENOFIBRATO EG         5   
62433  00176190429                          CIPRALEX         2   
62473  00176190429                    SERTRALINA SUN         2   
62836  00176190429                ACARBOSIO TECNIGEN         3   

               DATA_E_ORA  
24    2020-03-01 00:05:37  
229   2020-03-01 01:02:03  
542   2020-03-01 02:32:58  
884   2020-03-01 03:59:09  
1017  2020-03-01 04:38:03  
...                   ...  

02067420428
        PARTITAIVA                                   FARMACO  QUANTITA  \
32     02067420428                            NEBIVOLOLO SUN         3   
241    02067420428                                 DROPTIMOL         1   
573    02067420428                IRBESARTAN MACLEODS PHARMA         5   
702    02067420428                      RABEPRAZOLO TECNIGEN         1   
1022   02067420428                     NITROGLICERINA SANDOZ         4   
...            ...                                       ...       ...   
62039  02067420428                       PIOGLITAZONE ACCORD         1   
62176  02067420428                                   VISOFID         3   
62342  02067420428                   BICALUTAMIDE RATIOPHARM         2   
62608  02067420428  LISINOPRIL IDROCLOROTIAZIDE DOC GENERICI         5   
62719  02067420428                      LERCANIDIPINA GERMED         4   

               DATA_E_ORA  
32    2020-03-01 00:07:17  
241   2020-03-01 01:04:24  
573   2020-03-0

...

02632970428
        PARTITAIVA               FARMACO  QUANTITA          DATA_E_ORA
139    02632970428                EVISTA         1 2020-03-22 00:33:17
438    02632970428       LATANOPROST DOC         1 2020-03-22 01:59:02
461    02632970428          ENALAPRIL EG         4 2020-03-22 02:05:07
774    02632970428          ALGOPIRINDOL         5 2020-03-22 03:29:25
996    02632970428               TEXTAZO         3 2020-03-22 04:27:16
...            ...                   ...       ...                 ...
61312  02632970428               REMERON         3 2020-03-28 19:12:35
61661  02632970428  ACIDO IBANDRONICO EG         4 2020-03-28 20:02:43
62159  02632970428      TADALAFIL ACCORD         4 2020-03-28 21:26:34
62338  02632970428              MOSTAXYL         1 2020-03-28 22:13:54
62725  02632970428    ALLOPURINOLO MYLAN         3 2020-03-28 23:58:44

[268 rows x 4 columns]
10166250968
        PARTITAIVA                              FARMACO  QUANTITA  \
180    10166250968             

        PARTITAIVA                    FARMACO  QUANTITA          DATA_E_ORA
5      02159410428  FROVATRIPTAN DOC GENERICI         5 2020-03-22 00:01:06
338    02159410428                     OLEVIA         2 2020-03-22 01:29:14
468    02159410428          DUTASTERIDE MYLAN         1 2020-03-22 02:06:30
882    02159410428                     MEPRAL         4 2020-03-22 03:56:26
998    02159410428                     KEPPRA         5 2020-03-22 04:27:26
...            ...                        ...       ...                 ...
62025  02159410428          FLUOXETINA ARISTO         1 2020-03-28 20:53:11
62057  02159410428        ESOMEPRAZOLO GERMED         4 2020-03-28 20:59:03
62076  02159410428                   LUNIBRON         3 2020-03-28 21:03:26
62411  02159410428                      BIVIS         5 2020-03-28 22:36:03
62668  02159410428    MONTELUKAST TEVA ITALIA         4 2020-03-28 23:46:24

[277 rows x 4 columns]
02613310420
        PARTITAIVA                  FARMACO  QUANTIT

        PARTITAIVA                                FARMACO  QUANTITA  \
92     02269580425                          METFORMINA EG         4   
316    02269580425                         PREGABALIN SUN         5   
575    02269580425                               ZINADIUR         3   
758    02269580425                         IBUPROFENE DOC         1   
902    02269580425                  CEFTRIAXONE AUROBINDO         5   
...            ...                                    ...       ...   
61698  02269580425  AMOXICILLINA E ACIDO CLAVULANICO KRKA         3   
61822  02269580425                                  JUMEX         1   
62146  02269580425              EXEMESTANE MYLAN GENERICS         5   
62386  02269580425      IRBESARTAN IDROCLOROTIAZIDE MYLAN         5   
62549  02269580425                              VALBACOMP         5   

               DATA_E_ORA  
92    2020-03-22 00:22:51  
316   2020-03-22 01:23:45  
575   2020-03-22 02:36:14  
758   2020-03-22 03:26:01  
902   2

00135880425
        PARTITAIVA                   FARMACO  QUANTITA          DATA_E_ORA
155    00135880425            CITALOPRAM SUN         4 2020-03-22 00:36:50
247    00135880425                  IXILANIA         2 2020-03-22 01:05:29
652    00135880425                 SULIDAMOR         1 2020-03-22 02:56:43
878    00135880425   DULOXETINA DOC GENERICI         4 2020-03-22 03:55:22
968    00135880425          RAMIPRIL PHARMEG         1 2020-03-22 04:20:17
...            ...                       ...       ...                 ...
61793  00135880425                   LIXIDOL         5 2020-03-28 20:20:42
62048  00135880425                     NOLUS         1 2020-03-28 20:58:12
62065  00135880425  CIPROFLOXACINA AUROBINDO         1 2020-03-28 21:00:25
62396  00135880425                     ARAVA         4 2020-03-28 22:30:50
62612  00135880425              CELECOXIB EG         3 2020-03-28 23:29:02

[279 rows x 4 columns]
01577000423
        PARTITAIVA                       FARMACO  QU

00026370429
        PARTITAIVA                                  FARMACO  QUANTITA  \
150    00026370429                         RABEPRAZOLO TEVA         4   
377    00026370429                TOPIRAMATO MYLAN GENERICS         4   
641    00026370429                                DILATREND         1   
813    00026370429                  FINASTERIDE TEVA ITALIA         4   
1084   00026370429                       MEMANTINA TECNIGEN         5   
...            ...                                      ...       ...   
61403  00026370429  OLMESARTAN MEDOXOMIL E AMLODIPINA MYLAN         1   
62019  00026370429        PERINDOPRIL MYLAN GENERICS ITALIA         5   
62123  00026370429                                  STADIUM         5   
62487  00026370429                        FLUVASTATINA TEVA         4   
62660  00026370429                                  LAROXYL         2   

               DATA_E_ORA  
150   2020-03-22 00:35:51  
377   2020-03-22 01:40:13  
641   2020-03-22 02:53:43  

In [296]:
train_set = pd.concat([final_df_week_1, final_df_week_2, final_df_week_3], ignore_index = True)

In [297]:
train_set

,PARTITAIVA,WEEK_ID,1,2,3,4,5,6,7,8,...,159,160,161,162,163,164,165,166,167,168
0,02355240421,1.0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,-1,0,0,-1
1,00155670425,1.0,0,0,0,0,0,0,0,0,...,0,1,0,-1,1,-1,0,0,0,-1
2,02721500425,1.0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,-1,0,0,-1
3,03664141201,1.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-1,0,0,-1
4,00900100421,1.0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,02395430420,3.0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,-1,0,0,-1
662,01170410425,3.0,0,0,0,0,0,0,0,1,...,1,0,0,0,-1,1,-1,0,0,-1
663,01054480429,3.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-1,0,0,-1
664,00474070422,3.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1


In [298]:
for i in range(2,168):
    print('i =',i, train_set.iloc[:,i].unique())
    
for i in range(2,168):
    print('i =',i, final_df_week_4.iloc[:,i].unique())

i = 2 [0]
i = 3 [0]
i = 4 [0]
i = 5 [0]
i = 6 [0]
i = 7 [0]
i = 8 [0]
i = 9 [1 0]
i = 10 [ 0  1 -1]
i = 11 [0 1]
i = 12 [ 0 -1  1]
i = 13 [ 0 -1  1]
i = 14 [-1  0]
i = 15 [ 0  1 -1]
i = 16 [1 0]
i = 17 [ 0  1 -1]
i = 18 [ 0 -1  1]
i = 19 [ 0 -1  1]
i = 20 [ 0 -1  1]
i = 21 [ 0  1 -1]
i = 22 [-1  0 -2]
i = 23 [ 0 -1  1]
i = 24 [0 1]
i = 25 [0 1]
i = 26 [ 0 -1  1]
i = 27 [ 0  1 -1]
i = 28 [ 0 -1  1]
i = 29 [ 0  1 -1]
i = 30 [ 0 -1  1]
i = 31 [ 0  1 -1]
i = 32 [ 0 -1  1]
i = 33 [2 1 3]
i = 34 [ 0  1 -1]
i = 35 [1 0]
i = 36 [ 0  1 -1]
i = 37 [-1  0]
i = 38 [-1  0]
i = 39 [ 0  1 -1]
i = 40 [1 0 2]
i = 41 [0 1]
i = 42 [ 0 -1  1]
i = 43 [ 0 -1]
i = 44 [ 0  1 -1]
i = 45 [ 0 -1  1]
i = 46 [-2 -1]
i = 47 [ 0  1 -1]
i = 48 [ 1  0 -1]
i = 49 [-1  0  1]
i = 50 [ 1  0 -1]
i = 51 [-1  0  1]
i = 52 [ 0 -1  1]
i = 53 [ 0  1 -1]
i = 54 [ 1  0 -1]
i = 55 [-1  0  1]
i = 56 [ 0  1 -1]
i = 57 [2 1 3]
i = 58 [ 0  1 -1]
i = 59 [1 0]
i = 60 [ 0  1 -1]
i = 61 [-1  0]
i = 62 [-1  0 -2]
i = 63 [ 0 -1  1]
i = 64 [

In [230]:
svm = OneClassSVM(kernel='rbf', nu=2/222, gamma=0.000001)

In [231]:
svm.fit(train_set.iloc[:,2:])

OneClassSVM(gamma=1e-06, nu=0.009009009009009009)

In [232]:
y_pred = svm.predict(final_df_week_4.iloc[:,2:])
svm.score_samples(final_df_week_4.iloc[:,2:])

array([5.99991   , 5.99991   , 5.9999    , 5.999898  , 5.999884  ,
       5.99988   , 5.999916  , 5.9999    , 5.999916  , 5.9999    ,
       5.999908  , 5.99993   , 5.99992   , 5.99994   , 5.999896  ,
       5.999908  , 5.999858  , 5.999924  , 5.999942  , 5.999934  ,
       5.99991   , 5.999868  , 5.99987   , 5.999926  , 5.999918  ,
       5.999906  , 5.999916  , 5.999942  , 5.999942  , 5.99994   ,
       5.999926  , 5.99989   , 5.999926  , 5.999944  , 5.999928  ,
       5.9999    , 5.999904  , 5.999922  , 5.99960601, 5.999922  ,
       5.999922  , 5.999916  , 5.999918  , 5.999928  , 5.999894  ,
       5.99992   , 5.999918  , 5.999944  , 5.999914  , 5.999914  ,
       5.999922  , 5.99991   , 5.999894  , 5.999942  , 5.999924  ,
       5.9999    , 5.99992   , 5.999916  , 5.99992   , 5.999942  ,
       5.999906  , 5.999944  , 5.999918  , 5.99989   , 5.999906  ,
       5.999924  , 5.999882  , 5.999946  , 5.999918  , 5.999902  ,
       5.999918  , 5.999944  , 5.999912  , 5.999886  , 5.99994

In [214]:
y_pred

AttributeError: 'numpy.ndarray' object has no attribute 'score_samples'

In [209]:
final_df_week_4.sum().unique()

array(['02355240421001556704250272150042503664141201009001004210251681042700066360421012291204210130461042902769570421015414204260106329042100108460429015747404270250247042602549300420025521204260241241042100162290423009925604250280455042000045190428013046004200093658042200176190429000453604270251301042702542140427028036604280157704042901168600425003875504290206742042800171160427001563004280200684042102374750428001352004260231074042400817700420000947204220143088042502834420420023541404240019453042400156670424019779704310010641042600008080426013277704240003263042800935300426013250904290116543042001591190424025355404270003560042802310080425020774904290133116042201327200422011315704240072444042502354850428024840204210236474042902258310792011419504260229856042202404410421028327004270114879042902671260426012836604290009353042600005280425014310204270222064042501341060422028418304210269181042402540740426015731006800220683042002715180424027452704270001127042802204580423026810604280253123042901